# Evaluation of Time Parsings

In [1]:
from tf.app import use
import json
import nbformat as nbf
from pathlib import Path

with open('../../../results/parsing/time_parsings.json', 'r') as infile:
    time_parsings = json.load(infile)
    
# load TF
A = use('bhsa', hoist=globals())

A._browse = True

In [2]:
list(time_parsings.items())[0]

('427553',
 {'times': [2],
  'slots': [1, 2],
  'refs': [],
  'quals': ['point?'],
  'quants': [],
  'functions': ['simultaneous'],
  'preps': [['B', 1]]})

In [3]:
from pprint import pprint

In [27]:
import dictdiffer
import copy

In [28]:
test = copy.deepcopy(time_parsings['428448'])

In [112]:
test2=\
{'times': [{'times': [5148],
   'slots': [5148, 5148],
   'refs': [],
   'quals': ['point?'],
   'quants': [['NUMQ', 5148]],
   'functions': ['atelic_ext']},
  {'times': [5151, {'times': [1232, 12312]}],
   'slots': [5149, 5150, 5151],
   'refs': [['THE', 5150]],
   'quals': ['point?'],
   'quants': [],
   'functions': ['posterior'],
   'preps': [['>XR/', 5149]]}],
 'slots': [5148, 5148, 5149, 5150, 5151],
 'functions': ['posterior_dist', 'simul']}

test3 =\
{'times': [{'times': [5148],
   'slots': [5148, 5148],
   'refs': [],
   'quals': ['point?'],
   'quants': [['NUMQ', 5148]],
   'functions': ['atelic_ext']},
  {'times': [5151, {'times': [1232, 12312]}],
   'slots': [5149, 5150, 5151],
   'refs': [['THE', 5150]],
   'quals': ['point?'],
   'quants': [],
   'functions': ['posterior'],
   'preps': [['>XR/', 5149]]}],
 'slots': [5148, 5148, 5149, 5150, 5151],
 'functions': ['posterior_dist']}

In [110]:
diffs = list(
    dictdiffer.diff(
        test2, test3
    )
)

diffs

[('change', ['functions', 1], ('simul', 'sim'))]

In [113]:
dictdiffer.patch(diffs, test3)

IndexError: list assignment index out of range

In [108]:
[][0]

IndexError: list index out of range

In [50]:
print(
    json.dumps(diffs, indent='')   
)

[
[
"change",
[
"times",
1,
"times",
1,
"times",
0
],
[
{
"times": 1
},
1232
]
],
[
"add",
[
"times",
1,
"times",
1,
"times"
],
[
[
1,
12312
]
]
]
]


# Build Experimental Notebook

I'd like to consider using notebooks as a way to display time taggings 
and edit their code manually. Notebooks provide the best way to blend the 
code of the time parsings with the HTML of the Hebrew TF objects I'd like to
visualize for the evaluation process.

In [10]:
Path('bhsa.css').write_text(A.getCss())

11657

In [11]:
# Get translation data

with open('/Users/cody/github/CambridgeSemiticsLab/Gesenius_data/workflow/_private_/GBI_alignment/verse2text.json', 'r') as infile:
    english = json.load(infile)

In [12]:
english['esv']["('Genesis', 1, 1)"]

'In the beginning , God created the heavens and the earth .'

In [13]:
def mcell(content):
    return nbf.v4.new_markdown_cell(content)

def ccell(code):
    return nbf.v4.new_code_cell(code)

In [63]:
nb = nbf.v4.new_notebook()
cells = []

cells.append(
    mcell('# Time Adverbial Parsings (test)')
)
cells.append(ccell('''\
from eval_tools import load_style, Tracker
load_style()
tracker = Tracker('../../../results/parsing/time_parsings.json', 'corrections.json')
save = tracker.save\
'''))

nphrase = 0
for clause, parsing in time_parsings.items():
    
    if nphrase > 10:
        break
    
    clause = int(clause)
    ref = str(T.sectionFromNode(clause))
    pretty = A.pretty(clause, withNodes=True, highlights=parsing['slots'])
    parsecode = pprint.pformat(parsing)
    code = (
        f'save({clause},\n\n'
        f'{parsecode}\n\n'
        ')'
    )
    
    esv = english['esv'].get(ref, 'NOT AVAILABLE')
    display = f'{pretty}\nESV: {esv}'
    cells.append(
        mcell(display)
    )
    cells.append(
        ccell(code)
    )
    nphrase += 1
    
nb['cells'] = cells

nbf.write(nb, 'test.ipynb')